CSCM27 Assignment 1 By Oliver Barnes 1905121

In [2]:
import pandas as pd
import altair as alt

In [213]:
# data initially used was in .csv format, however Altair was acting very strangely
alt.data_transformers.enable('json')
url = 'heart-disease.json'

df = pd.read_json(url)
df.head()

# some values (like sex) are encoded as boolean

# cpt = chest pain type
# restbp = resting blood pressure
# chol = cholesterol
# fbs = fasting blood sugar (>120 mg/dl)
# restecg = resting electrocardiographic results (0 = normal)
# maxhr = maximum heart rate achieved during excercise
# oldpeak = ST depression induced by exercise relative to rest (?)
# slope = the slope of the peak exercise ST segment (?)
# thal = 3 = normal; 6 = fixed defect; 7 = reversable defect (result of thallium test?)


,age,sex,cpt,restbp,chol,fbs,restecg,maxhr,excercise-induced,oldpeak,slope,major-vessels-coloured,thal,diagnosis
0,63,Male,1,145,233,1,2,150,0,2.3,3,0,6,Negative
1,67,Male,4,160,286,0,2,108,1,1.5,2,3,3,Positive
2,67,Male,4,120,229,0,2,129,1,2.6,2,2,7,Positive
3,37,Male,3,130,250,0,0,187,0,3.5,3,0,3,Negative
4,41,Female,2,130,204,0,2,172,0,1.4,1,0,3,Negative


In [282]:
# Final Implementation

# defining selection for interaction
singlePoint = alt.selection_single(on='mouseover', fields=['diagnosis'], nearest=True)

# calculation of CRV must be done independant of everything else
chart = alt.LayerChart(df).transform_calculate(
    crv='datum.age * (datum.restbp/100) * (datum.chol/100) * (1/datum.maxhr)')

# definitions for charts
# allows for easy concatenation
chartCombi = chart.properties(
    title=(['Heart Disease: Risk against diagnoses', 'Combined'])
)
chartM = chart.transform_filter('datum.sex == "Male"').properties(
    title='Male Only'
)
chartF = chart.transform_filter('datum.sex == "Female"').properties(
    title='Female Only'
)

# common factors shared by both point and line charts
# X, Y, and colour should be the same for all 3
common = alt.Chart(df).encode(
    alt.X('crv:Q', bin=alt.BinParams(maxbins=20), title="Compound Risk Value"),
    alt.Y('count()', scale=alt.Scale(domain=[1, 40]), title="Number of Diagnoses"),
    alt.Color('diagnosis:N', legend=alt.Legend(title="Diagnosis"))
)

# definition for scatter plot
points = common.mark_point().encode(
    tooltip=['count()'],
    size=alt.condition(singlePoint, alt.value(40), alt.value(20))
).add_selection(
    singlePoint
)

# definition for line graph
line = common.mark_line().encode(
    size=alt.condition(singlePoint, alt.value(3), alt.value(1))
)

# Facet would break the plot, but with concatenation it outputs fine
chartCombi + points + line & chartM + points + line & chartF + points + line

alt.VConcatChart(...)